### SCRAPER DO BRICKSET

Tutaj kod niestety nie odpali się elegancko - żeby wszystko się ładnie zebrało, musiałam pobrać dane indexów setów z innej strony,
w międzyczasie pobrałam trochę setów innymi metodami. Ostatecznie musiałam zrobić listę zawierającą różnicę setów które już posiadam, 
a które jeszcze muszę mieć. Te dwie listy miałam w plikach csv na dysku.

Kiedy miałam już wszystkie pliki CSV załadowałam je z poziomu Excela przez Power Query w jeden plik - dlatego nie ma później formuły w Pythonie, która je łączy.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Ścieżki do plików CSV na dysku. Tutaj mam sety które udało mi zgrać oraz które muszę jeszcze zgrać.
scrape_csv_path = 'SCRAPE.csv'
got_csv_path = 'GOT.csv'

scrape_df = pd.read_csv(scrape_csv_path, sep=';', error_bad_lines=False, warn_bad_lines=True)
got_df = pd.read_csv(got_csv_path, sep=';', error_bad_lines=False, warn_bad_lines=True)

new_sets = scrape_df['set_num'].tolist()
got_sets = got_df['Number'].tolist()

# To jest lista zawierającą numery z listy 'do pobrania', które nie powtarzają się w liście "już pobrane"
d_sets = [num for num in new_sets if num not in got_sets]

print(len(new_sets))
print(len(got_df))
print(len(new_sets))
print(len(d_sets))

In [3]:
# Funkcja do scrapowania danych z Brickset

def scrap_page(url, max_retries=5, backoff_factor=60):
    attempts = 0
    while attempts < max_retries:
        response = requests.get(url)
        
        if response.status_code == 200:

            soup = BeautifulSoup(response.content, 'html.parser')
            text_div = soup.find('div', class_='text')
            
            if text_div:
                dt_elements = text_div.find_all('dt')
                dd_elements = text_div.find_all('dd')

                data = {'Number':'','Name':'','Type':'','Theme group':'','Theme':'','Year released':'', 'Launch/exit': '','Pieces':'','Minifigs':'','Designer':'','RRP': '', 'Age range':'','Packaging':'','Weight':'','Availability':''}

                for dt, dd in zip(dt_elements, dd_elements):
                    dt_text = dt.get_text(strip=True)
                    dd_text = dd.get_text(strip=True)

                    if dt_text == 'Number':
                        data['Number'] = dd_text
                    elif dt_text == 'Name':
                        data['Name'] = dd_text
                    elif dt_text == 'Type':
                        data['Type'] = dd_text
                    elif dt_text == 'Theme group':
                        data['Theme group'] = dd_text 
                    elif dt_text == 'Theme':
                        data['Theme'] = dd_text
                    elif dt_text == 'Year released':
                        data['Year released'] = dd_text 
                    elif dt_text == 'Launch/exit':
                        data['Launch/exit'] = dd_text 
                    elif dt_text == 'Pieces':
                        data['Pieces'] = dd_text 
                    elif dt_text == 'Minifigs':
                        data['Minifigs'] = dd_text
                    elif dt_text == 'Designer':
                        data['Designer'] = dd_text 
                    elif dt_text == 'RRP':
                        data['RRP'] = dd_text 
                    elif dt_text == 'Age range':
                        data['Age range'] = 'x' + str(dd_text)
                    elif dt_text == 'Packaging':
                        data['Packaging'] = dd_text
                    elif dt_text == 'Weight':
                        data['Weight'] = dd_text 
                    elif dt_text == 'Availability':
                        data['Availability'] = dd_text 

                return data
            else:
                print(f"No 'text' div found on the page {url}")
                return None
        elif response.status_code == 429:
            print(f"Received 429 Too Many Requests. Retrying in {backoff_factor} seconds...")
            time.sleep(backoff_factor)
            attempts += 1
        else:
            print(f"Failed to retrieve the page {url}. Status code: {response.status_code}")
            return None
    print(f"Max retries reached for {url}")
    return None


# Pobierałam dane batchami - a) żeby strona mi zablokowała I.P. i b) często wykrzaczało się z innych powodów, internet czasem padał
num_rows_to_scrape = 1000

# Zawsze wpisywałam od nowa index batcha, który mam teraz ściągać
start_index = 6000

all_data = []

# Scrapowanie danych ze wszystkich stron z listy d_sets, zaczynając od start_index
for set_num in d_sets[start_index:]:
    if len(all_data) >= num_rows_to_scrape:
        break
    page_url = f"https://brickset.com/sets/{set_num}"
    print(f"Scrapowanie strony: {page_url}")
    data = scrap_page(page_url)
    if data:
        all_data.append(data)
    
    time.sleep(2)

df = pd.DataFrame(all_data)

# Zapisuję dany batch do csv - oznaczam zakres batcha w nazwie
df.to_csv('scrapowane_dane_6000_7000.csv', index=False)

print("Dane zostały zapisane do pliku 'scrapowane_dane.csv'")

Scrapowanie strony: https://brickset.com/sets/5006010-1
Scrapowanie strony: https://brickset.com/sets/5006012-1
No 'text' div found on the page https://brickset.com/sets/5006012-1
Scrapowanie strony: https://brickset.com/sets/5006013-1
No 'text' div found on the page https://brickset.com/sets/5006013-1


KeyboardInterrupt: 